In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from astropy.modeling import fitting, models
from scipy.integrate import quadrature,trapz
import seaborn as sb
import os
from io import BytesIO as bytesio
from ipywidgets import widgets
import ipywidgets as ipw

sb.set_style('darkgrid')
matplotlib.rcParams['font.size']=12
matplotlib.rcParams['figure.figsize']=(10,10)

# **Exposure Time Calculator**

In [2]:
def coeff_calc(x0,xn,x=None,y=None,mode = None):
    if mode == 'Gaussian':
        model = models.Gaussian1D(mean = (x0+xn)*0.5, stddev = xn-x0)
        int_y,err = quadrature(model,x0,xn)
        return int_y/(xn-x0)
    else :
        return trapz(y,x)/(xn-x0)

In [3]:
def exposure_time(det_params,M,SNR):
    wavelength     = det_params['wavelength']
    bandwidth      = det_params['bandwidth']
    effective_area = det_params['effective_area']
    M_sky          = det_params['sky_brightness']
    plate_scale    = det_params['plate_scale']
    aperture       = det_params['aperture']
    dark_current   = det_params['dark_current']
    read_noise     = det_params['read_noise']

    F_0_p   = 1.51e3*(bandwidth/wavelength)*3631*effective_area
    F_m_p   = F_0_p*pow(10,-0.4*M)
    M_sky_p = M_sky - 2.5*np.log10(plate_scale**2)
    F_sky_p = F_0_p*pow(10,-0.4*M_sky_p)

    n_pix   = np.pi*((0.5*aperture)/plate_scale)**2

    A =  (F_m_p/SNR)**2
    B = -(F_m_p + F_sky_p*n_pix + dark_current*n_pix)
    C = -n_pix*(read_noise)**2

    t1 = (-B + np.sqrt(B**2 - 4*A*C))/(2*A)
    t2 = (-B - np.sqrt(B**2 - 4*A*C))/(2*A)
    t = np.where(t1>0,t1/60,t2/60)

    return t

In [4]:
def SNR_t(det_params,M,SNR):
    wavelength     = det_params['wavelength']
    bandwidth      = det_params['bandwidth']
    effective_area = det_params['effective_area']
    M_sky          = det_params['sky_brightness']
    plate_scale    = det_params['plate_scale']
    aperture       = det_params['aperture']
    dark_current   = det_params['dark_current']
    read_noise     = det_params['read_noise']

    F_0_p   = 1.51e3*(bandwidth/wavelength)*3631*effective_area
    F_m_p   = F_0_p*pow(10,-0.4*M)
    M_sky_p = M_sky - 2.5*np.log10(plate_scale**2)
    F_sky_p = F_0_p*pow(10,-0.4*M_sky_p)

    n_pix   = np.pi*((0.5*aperture)/plate_scale)**2

    B = (F_m_p + F_sky_p*n_pix + dark_current*n_pix)
    C = n_pix*(read_noise)**2
    t_out =  []
    t     = 1e-2
    dt    = 1e-1
    
    SNR_out = []
    SNR_t   = 0
    while True:
        SNR_t =  (F_m_p*t)/(np.sqrt(B*t +C))
        if SNR_t>SNR or len(t_out)>1000:
            break
        t_out.append(t)
        SNR_out.append(SNR_t)
        t +=dt  
        
    return t_out,SNR_out

In [5]:
df = pd.read_csv('data/detector_parameters.csv')

In [6]:
def pixel_scale(src):
    choices['telescope'] = src
    return str(df[df['telescope']==src]['plate_scale'].values[0])

In [7]:
def filters(src):
    eff_l.value = r"Effective area $(cm^2)$ (From Database)"
    return df[df['telescope']==src]['filter'].values

In [8]:
def bandwidth(src):
    choices['filter'] = src
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['bandwidth'].values[0])

In [9]:
def eff_lambda(src):
    eff_l.value = r"Effective area $(cm^2)$ (From Database)"
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['wavelength'].values[0])

In [10]:
def eff_area(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['effective_area'].values[0])

In [11]:
def aperture(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['aperture'].values[0])

In [12]:
def rn(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['read_noise'].values[0])

In [13]:
def dc(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['dark_current'].values[0])

In [14]:
def sky_bri(src):
    return str(df[(df['telescope']==choices['telescope']) & (df['filter']==src)]['sky_brightness'].values[0])

In [15]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html_buttons = '''<html>
        <head>
        <meta name="viewport" content="width=device-width, initial-scale=1">
        </head>
        <body>
        <a download="{filename}" href="data:text/csv;base64,{payload}" download>
        <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download as CSV</button>
        </a>
        </body>
        </html>
        '''
    html_button = html_buttons.format(payload=payload,filename=filename)
    return display(HTML(html_button))
    

In [20]:
%%html
<style>
.box_style1{
    width:auto;
    border : 2px solid black;
    height: auto;
    background-color:#FF5233 ;
}
</style>


In [21]:
%%html
<style>
.box_style2{
    width:auto;
    border : 2px solid black;
    height: auto;
    background-color:#FFA833;
}
</style>


In [22]:
%%html
<style>
.box_style3{
    width:auto;
    border : 2px solid black;
    height: auto;
    background-color:#FF5233 ;
}
</style>


In [33]:
global choices
choices = {}
choices['coeff'] = None

eff_l = widgets.Label(r"Effective area $(cm^2)$ (From Database)")
eff_t = widgets.Text("0.9")

#------------------Telescope-------------------

c1 = []
c1.append(widgets.HTML("<h2>----------------Telescope-----------------</h2>"))
c1.append(widgets.Label("Select Telescope"))

c1.append(widgets.Dropdown(
    options=df['telescope'].unique(),
    value='INSIST'))

c1.append(widgets.HTML('<h>Diameter of primary (cm)</h>'))
c1.append(widgets.Text("100"))

c1.append(widgets.HTML('<h>Plate Scale (\"/pixels)</h>'))
c1.append(widgets.Text())

c1.append(widgets.HTML('<h>No of reflecting surfaces</h>'))
c1.append(widgets.Text("5"))

c1.append(widgets.HTML('<h>Coating coefficient</h>'))
c1.append(widgets.Text("0.8"))

c1.append(widgets.HTML("<h fontsize=10>Custom coating coefficient</h>"))
c1.append(widgets.FileUpload(multiple=True))

ipw.dlink((c1[2],'value'), (c1[6],'value'), transform = pixel_scale)

v1 = widgets.VBox(c1,layout=ipw.Layout(justify_content= 'flex-end'))

v1.add_class("box_style1")
#------------------Filter-------------------------

c2 = []

c2.append(widgets.HTML("<h2>---------------------Filter--------------------<h2>"))
c2.append(widgets.Label("Select Filter :"))

c2.append(widgets.Dropdown(
    options=df[df['telescope']==c1[2].value]['filter'].unique(),
    value='UV'))

ipw.dlink((c1[2],'value'), (c2[2],'options'), transform = filters)

c2.append(widgets.Label(r"$\Delta\lambda$ (Å)"))
c2.append(widgets.Text())

c2.append(widgets.Label(r"$\lambda_{eff}$ (Å)"))
c2.append(widgets.Text())

c2.append(widgets.Label("Transmission Coefficient"))
c2.append(widgets.Text("0.9"))

c2.append(widgets.Label("Sky Brightness"))
c2.append(widgets.Text())


c2.append(widgets.HTML("<h fontsize=10>Custom filter Response functions</h>"))
c2.append(widgets.FileUpload(multiple=True))

ipw.dlink((c2[2],'value'), (c2[4],'value'), transform = bandwidth)
ipw.dlink((c2[2],'value'), (c2[6],'value'), transform = eff_lambda)
ipw.dlink((c2[2],'value'), (c2[10],'value'), transform = sky_bri)
ipw.dlink((c2[2],'value'), (eff_t,'value'), transform = eff_area)

v2 = widgets.VBox(c2,layout=ipw.Layout(justify_content= 'flex-end'))
v2.add_class("box_style2")
#--------------------Detector----------------------------

c3 = []

c3.append(widgets.HTML("<h2>----------------Detector----------------</h2>"))
c3.append(widgets.Label("Photometric aperture (arcsec)"))
c3.append(widgets.Text())

ipw.dlink((c2[2],'value'), (c3[2],'value'), transform = aperture)

c3.append(widgets.Label(r"Quantum efficiency"))
c3.append(widgets.Text("0.5"))

c3.append(widgets.Label("Read Noise"))
c3.append(widgets.Text())

c3.append(widgets.Label("Dark Current"))
c3.append(widgets.Text())

c3.append(widgets.HTML("<h fontsize=10>Custom Quantum Efficiency profile</h>"))
c3.append(widgets.FileUpload(multiple=True))

ipw.dlink((c2[2],'value'), (c3[6],'value'), transform = rn)
ipw.dlink((c2[2],'value'), (c3[8],'value'), transform = dc)

v3 = widgets.VBox(c3)

v3.add_class("box_style3")

h = widgets.HBox([v1, v2,v3])


display(h)
display(eff_l,eff_t)

eff_b = widgets.Button(description="Recalculate")

display(eff_b)

h0 = widgets.HTML("<h1>Source</h1>")
display(h0)

l1 = widgets.Label("Select calculator mode")
display(l1)

mode = widgets.Dropdown(
    options=['Exposure time','SNR'],
    value='Exposure time'
)
display(mode)

l2 = widgets.Label("Enter AB magnitude of the source")
display(l2)
Mag = widgets.Text("20")
display(Mag)

l2 = widgets.Label("Enter required SNR")
display(l2)

SNR = widgets.Text("5")
display(SNR)

btn = widgets.Button(description="Submit")

display(btn)

output = widgets.Output()

@output.capture(clear_output=True,wait=True)

def recalc(b):
    # Telescope coefficients
    if len(c1[-1].value)> 0 :
        coeff_t = []
        for file in c1[-1].value:
            df_coff = pd.read_csv(bytesio(c1[-1].value[file]['content']))
            coeff_t.append(coeff_calc(df_coeff['wav'].min(),df_coeff['wav'].max(),x=df_coeff['wav'].values,y=df_coeff['response'].values))
        coeff_t = np.array(coeff_t).prod()
        no_refl_surf = float(c[8].value)

        coeff_t = pow(coeff_t,no_refl_surf)

    elif c1[10].value.replace('.','',1).isnumeric()==True and float(c1[10].value)>0 and c1[8].value.replace('.','',1).isnumeric()==True:
        coeff_t = float(c1[10].value)
        no_refl_surf = float(c1[8].value)
        coeff_t = pow(coeff_t,no_refl_surf)
        
    # Filter Coefficients

    if len(c2[-1].value)> 0 :
        coeff_t = []
        for file in c1[-1].value:
            df_coff = pd.read_csv(bytesio(c1[-1].value[file]['content']))
            coeff_f.append(coeff_calc(df_coeff['wav'].min(),df_coeff['wav'].max(),x=df_coeff['wav'].values,y=df_coeff['response'].values))

        coeff_f = np.array(coeff_f).prod()

    elif c2[8].value.replace('.','',1).isnumeric()==True :
        if float(c2[8].value)>0:
            coeff_f = float(c2[8].value)

    # Detecter Coefficients
    if len(c3[-1].value)> 0 :
        coeff_d = []
        for file in c1[-1].value:
            df_coff = pd.read_csv(bytesio(c1[-1].value[file]['content']))
            coeff_d.append(coeff_calc(df_coeff['wav'].min(),df_coeff['wav'].max(),x=df_coeff['wav'].values,y=df_coeff['response'].values))  
        coeff_d = np.array(coeff_d).prod()
    elif c3[4].value.replace('.','',1).isnumeric()==True :
        if float(c3[4].value)>0:
            coeff_d = float(c3[4].value)
    
    choices['coeff']          = coeff_t*coeff_f*coeff_d 
    choices['effective_area'] = choices['coeff']*0.25*np.pi*float(c1[4].value)**2 
    
    eff_l.value = r"Effective area $(cm^2)$ (Calculated)"
    eff_t.value = f"{choices['effective_area']}"
    
@output.capture(clear_output=True,wait=True)

def submit(b):
    if len(Mag.value)<1:
        Mag.value = '22'
        out = widgets.Label(f"Default AB magnitude set : 22")
        display(out)
    if len(SNR.value)<1:
        SNR.value = '5'
        out = widgets.Label(f"Default SNR set : 5")
        display(out)

    if float(Mag.value)>0  and float(SNR.value)>0 :
        
        temp = df[ (df['telescope']==choices['telescope']) & (df['filter']==choices['filter'])]

        global det_params

        det_params = choices
        det_params['wavelength']     = float(c2[6].value) if c2[6].value.replace('.','',1).isnumeric()==True else temp['wavelength'].values[0]
        det_params['bandwidth']      = float(c2[4].value) if c2[4].value.replace('.','',1).isnumeric()==True else temp['bandwidth'].values[0]

        if len(eff_t.value)>0:
            det_params['effective_area'] =  float(eff_t.value)

        det_params['sky_brightness'] = float(c2[10].value) if c2[10].value.replace('.','',1).isnumeric()==True else temp['sky_brightness'].values[0]
        det_params['plate_scale']    = float(c1[6].value) if c1[6].value.replace('.','',1).isnumeric()==True else temp['pixel_scale'].values[0]
        det_params['aperture']       = float(c3[2].value) if c3[2].value.replace('.','',1).isnumeric()==True else temp['aperture'].values[0]
        det_params['read_noise']     = float(c3[6].value) if c3[6].value.replace('.','',1).isnumeric()==True else temp['read_noise'].values[0]
        det_params['dark_current']   = float(c3[8].value) if c3[8].value.replace('.','',1).isnumeric()==True else temp['dark_current'].values[0]

        if mode.value=="Exposure time" and float(SNR.value)>0:
            t = exposure_time(det_params,float(Mag.value),float(SNR.value))
            out = widgets.Label(f"Required exposure time for observing source with magnitude {Mag.value} and SNR {SNR.value} = {np.round(t,5)} minutes")
            display(out)
            l1 = widgets.Label(f"Effective Area used in calclation : {np.round(det_params['effective_area'],3)} cm2")
            display(l1)
            
        elif mode.value=="SNR":
            t,SNR_ = SNR_t(det_params,float(Mag.value),float(SNR.value))
            plt.figure(figsize = (15,8))
            plt.plot(t,SNR_,'.-')
            
            df_out = pd.DataFrame(zip(t,SNR_), columns=['exp_time','SNR'])
                                  
            plt.xlabel('time (seconds)')
            plt.ylabel('SNR')
            plt.title(f'Exposure time Vs SNR for source of ABmag : {Mag.value}')
            plt.show()
               
            create_download_link(df_out)
        else:
            out = widgets.Label("Check Input")
            display(out)
            
    else:
        out = widgets.Label("AB magnitude and SNR should be greater than 0")
        display(out)

btn.on_click(submit)
eff_b.on_click(recalc)

output

Label(value='Effective area $(cm^2)$ (From Database)')

Text(value='1042.30504')

Button(description='Recalculate', style=ButtonStyle())

HTML(value='<h1>Source</h1>')

Label(value='Select calculator mode')

...

Text(value='20')

Label(value='Enter required SNR')

Text(value='5')

Button(description='Submit', style=ButtonStyle())

Output()